In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from warnings import simplefilter,filterwarnings
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
import matplotlib.ticker as mticker
import matplotlib.ticker as ticker
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import scipy.io
import pickle
import sys
import gc
import os
import re
filterwarnings('ignore')
filterwarnings(action='once')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
simplefilter('ignore', FutureWarning)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
root_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS'
code_path = os.path.join(root_path, 'Codes','Shared Codes')
data_path =  os.path.join(root_path, 'data')
results_path = os.path.join(root_path, 'Results')

# Results of GP - MSANA
VLPSO_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/Results of GP - MSANA'
folder_name = 'VLPSO '
filenames = ['ISCX2012', 'CSE-CIC2018', 'CICIDS2017', 'kddcup99_csv', 'sorted_ALLFLOWMETER_HIKARI2021']
files = list(map(lambda x: os.path.join(VLPSO_path, "MSANA__"+x), filenames))
sys.path.insert(0,code_path)
from ensemble import Ensemble, Classifier

In [7]:
def save_object(obj, filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()
def load_object(filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [ ]:
for i, f in enumerate(files):
  scores = {}
  old_result = load_object(filenames[i] + ".csv_ensemble", os.path.join(os.path.join(results_path, 'kpfaoselm_gp_combiner_with_model_replacement_evolving_DFS'), filenames[i]+".csv")).scores
  new_results = load_object('result_dic', f)
  y_true, y_pred = [], []
  for key in new_results['y_test'].keys():
    y_true = y_true + new_results['y_test'][key]
    y_pred = y_pred + new_results['y_pred'][key]
  number_of_chunk = len(old_result.keys())
  chunks_preds = np.array_split(y_pred, number_of_chunk)
  chunks_labels = np.array_split(y_true, number_of_chunk)
  chunk_number = 1
  for CN,chunk_X, chunk_Y in tqdm(zip([*range(len(chunks_labels))],chunks_preds, chunks_labels)):
    try:
      auc = roc_auc_score(chunk_Y, chunk_X)
    except:
      auc = 0.5
    scores[chunk_number] = {"accuracy": accuracy_score(chunk_Y, chunk_X),
                           "precision": precision_score(chunk_Y, chunk_X),
                            "recall": recall_score(chunk_Y, chunk_X),
                            "f1-score": f1_score(chunk_Y, chunk_X),
                            "auc": auc}
    chunk_number += 1
  path = os.path.join(os.path.join(results_path, folder_name), filenames[i]+".csv")
  if not os.path.exists(path): os.makedirs(path)
  save_object(scores, filenames[i] + ".csv_ensemble", path)
  print(load_object(filenames[i] + ".csv_ensemble", path).keys())
